In [ ]:
# Переклад на іншу мову текста книги формата 
# epub за допомогою моделі нейронної мерережі
# яка використовує архітектуру Transformer
# https://github.com/UKPLab/EasyNMT, подивіться що потрібно встановити для роботи моделі
# pip install ebooklib встановіть для роботи з книгами
# pip install bs4 для роботи з сторінками книг
# ідея виникла після знайомства з текстами програми https://github.com/yihong0618/bilingual_book_maker

from easynmt import EasyNMT
from ebooklib import  epub, ITEM_DOCUMENT,ITEM_STYLE,ITEM_NAVIGATION
from bs4 import BeautifulSoup as bs

In [ ]:
# вкажіть мову, з якої треба перекласти, на яку, і шлях до книги
# в вихідній книзі до імені файла додастся мова перекладу
# переклад записужться в книгу по параграфам і його завжди після
# переривання процесу можна продовжити. Перекладені параграфи помічаються
# атрибутом done

# на яку мову
target_lang="ru" 
# з якої
source_lang="en"
# шлях до книги
source_path="Вкажіть шлях до книги"
target_path=source_path.rsplit('.epub')[0]+'_'+target_lang+'.epub'

In [ ]:
# функції

# не перекладати елемент тегу
def no_translate(item):
    return False
    no_translate_tags=['code']
    if item.name:
        return item.name in no_translate_tags
    else:
        return False
    
# зберегти книгу в target_path    
def save_book(book):
    epub.write_epub(target_path, book, {})

# обєднати два елементи тегу, які є одним словом
def together(first,second):
    if not second.name:
        second.replace_with(first.text+second.text)
        return   
    for item in second.contents:
        if item.name:
            together(first,item)
        else:
            item.replace_with(first.text+item.text)
            return
# об'єднати елементи тегу, які є одним словом        
def prepare(con):
   if len(con)>1: 
      for i in range(len(con)-1):
       if no_translate(con[i]):
            continue
       if con[i].text and not con[i].text[len(con[i].text)-1].isspace() and  con[i+1].text and not con[i+1].text[0].isspace():
            together(con[i],con[i+1])
            con.remove(con[i])
            if len(con)==1:
                return
            prepare(con)
            return 
   elif con:
      if con[0].name:
        prepare(con[0].contents)
            
# перекласти вміст тегу
def translate(con):
    for i in con:
        if no_translate(i):
            continue
        if i.name:
            translate(i)
        else:    
            i.replace_with(model.translate(i.text,target_lang=target_lang,source_lang=source_lang))

# перекласти метадані книги            
def translate_metadata(book):
    book.title = model.translate(book.title,target_lang=target_lang,source_lang=source_lang)
    book.language = target_lang
    
    metadata = book.get_metadata('DC','title' )
    metadata_text=model.translate(metadata[0][0],target_lang=target_lang,source_lang=source_lang)
    if isinstance(metadata_text,bytes):
        metadata_text=metadata_text.decode('utf-8')
    book.set_unique_metadata(name='title',value=metadata_text,namespace='DC')
    
    metadata = book.get_metadata('DC','creator' )
    metadata_text=model.translate(metadata[0][0],target_lang=target_lang,source_lang=source_lang)
    if isinstance(metadata_text,bytes):
        metadata_text=metadata_text.decode('utf-8')
    book.set_unique_metadata(name='creator',value=metadata_text,namespace='DC')
'''   
    metadata = book.get_metadata('DC','description' )
    soup=bs(metadata[0][0], "xml")
    prepare
    translate(soup.contents)
    metadata_text=soup.prettify().encode()  
    if isinstance(metadata_text,bytes):
         metadata_text=metadata_text.decode('utf-8')
    book.set_unique_metadata(name='description',value=metadata_text,namespace='DC')
'''

# перекласти всі сторінки книги
def translate_pages(book):
    items=[item for item in book.get_items() if item.get_type()==ITEM_DOCUMENT]

    for item in items:
        for l in bs(item.content,'xml').findAll('link'):
            item.add_link(href=l['href'],rel=l['rel'],type=l['type'])
        
    for p, item in enumerate(items):
        print('page:',p, end='   ')
        soup=bs(item.content, "xml")
        tag_list=[tag for tag in soup.find('body').contents if tag.name and  tag.text and not tag.has_attr('done')]
        for tag in tag_list:
            if no_translate(tag):
                continue
            prepare(tag.contents)
            translate(tag.contents)
            print(len(tag.text), sep=', ',end=' ')
            tag.attrs["done"]="yes"
            item.content=soup.prettify().encode()
            save_book(book)
        print()
                  


In [ ]:
#Завантажити модель перекладу, розкоментуйте потрібну і закоментуйте всі інші 

#model = EasyNMT('opus-mt') # Helsinki NLP 186 мов 300 Мб, найшвидша
#model = EasyNMT('mbart50_m2m') #Facebook Research  52 мови, 2.3 Гб
#model = EasyNMT('mbart50_en2m') #Facebook Research  52 мови, з англійської на інші 2.3 Гб
model = EasyNMT('m2m_100_418M') # Facebook Research  100 мов 1.8 Гб  
#model = EasyNMT('m2m_100_1.2B') # Facebook Research  100 мов 5 Гб  

In [ ]:
# перекласти книгу 
book=epub.read_epub(source_path)
translate_metadata(book)
translate_pages(book)

In [ ]:
# продовжити переклад книги 
book=epub.read_epub(target_path)
translate_pages(book)